# Telco Customer Churn Analysis
Dataset- [https://www.kaggle.com/datasets/blastchar/telco-customer-churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

## Task

Perform Transform:
 
**✔ Cleaning Tasks**
- Convert "**TotalCharges**" to numeric (dataset has spaces → become **NaN**).
- Fill missing numeric values using:
    - Median for **tenure**, **MonthlyCharges**, **TotalCharges**.
    - Replace missing categorical values with "**Unknown**".

<br>

**✔ Feature Engineering**

Create the following new columns:
1. **tenure_group**
Based on tenure months:
0–12   → "*New*"
13–36  → "*Regular*"
37–60  → "*Loyal*"
60+    → "*Champion*"
2. **monthly_charge_segment**
MonthlyCharges < 30  → "*Low*"
30–70              → "*Medium*"
\> 70                 → "*High*"
3. **has_internet_service**
Convert InternetService column:
"DSL" / "Fiber optic" → 1
"No" → 0
4. **is_multi_line_user**
1 if MultipleLines == "Yes"
0 otherwise
5. **contract_type_code**
Map:
Month-to-month → 0
One year      → 1
Two year      → 2

<br>

**✔ Drop unnecessary fields**

Remove:
customerID, gender

In [70]:
# imports
import pandas as pd
import numpy as np

## Load Data

In [71]:
path = ".\..\Data Scource\WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(path)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data Inspection

In [72]:
df.shape

(7043, 21)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [74]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [75]:
# Null values analysis
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

## Data Cleaning

### Data type conversion

In [76]:
# map spaces to NaN
df['TotalCharges'] = df['TotalCharges'].replace(r'\s+', np.nan, regex=True)

In [77]:
# data type conversion
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

### Handling missing values

In [78]:
# tenure - median value
df.tenure.median()

29.0

In [79]:
df['tenure'] = df['tenure'].fillna(df['tenure'].median())

In [80]:
# MonthlyCharges - median value
df.MonthlyCharges.median()

70.35

In [81]:
df['MonthlyCharges'] = df['MonthlyCharges'].fillna(df['MonthlyCharges'].median())

In [82]:
# TotalCharges - median value
df.TotalCharges.median()

1397.475

In [83]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

In [84]:
# Categorical values - fill with UNKNOWN
df['SeniorCitizen'] = df['SeniorCitizen'].fillna('UNKNOWN')

## Feature Engineering

In [85]:
# tenure_group
df['tenure_group'] = pd.cut(df['tenure'], bins=[0, 12, 36, 60, np.inf], labels=['New', 'Regular', 'Loyal', 'Champion'])

In [86]:
# monthly_charge_segment
df['monthly_charge_segment'] = pd.cut(df['MonthlyCharges'], bins=[0, 30, 70, np.inf], labels=['Low', 'Med', 'High'])

In [87]:
# has_internet_service
df['has_internet_service'] = np.where((df['InternetService'] == 'DSL') | (df['InternetService'] == 'Fiber optic'), 1, 0)

In [88]:
# is_multi_line_user
df['is_multi_line_user'] = np.where(df['MultipleLines'] == 'Yes', 1, 0)

In [89]:
# contact_type_code
df['contact_type_code'] = df['Contract'].map({
    'Month-to-month': 0,
    'One year': 1,
    'Two year': 2
})

## Dropping unneccessary fields

In [90]:
# customerID, gender
df.drop(['customerID', 'gender'], axis=1, inplace=True)

In [91]:
df.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group,monthly_charge_segment,has_internet_service,is_multi_line_user,contact_type_code
0,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,...,Yes,Electronic check,29.85,29.85,No,New,Low,1,0,0
1,0,No,No,34,Yes,No,DSL,Yes,No,Yes,...,No,Mailed check,56.95,1889.50,No,Regular,Med,1,0,1
2,0,No,No,2,Yes,No,DSL,Yes,Yes,No,...,Yes,Mailed check,53.85,108.15,Yes,New,Med,1,0,0
3,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,...,No,Bank transfer (automatic),42.30,1840.75,No,Loyal,Med,1,0,1
4,0,No,No,2,Yes,No,Fiber optic,No,No,No,...,Yes,Electronic check,70.70,151.65,Yes,New,High,1,0,0
